### Importing the Libraries 

In [834]:
from bs4 import BeautifulSoup as bs
import lxml
import xml.etree.ElementTree as ET
from random import choice
import random 
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [835]:
RUNS = 5 #number of time an experiment should run

In [836]:
#A custom exception class to handle the termination of a run based on number of fitness evaluations
class TerminationException(Exception):
    pass

### Parse the Burma file extract the contents

In [837]:
# Parse the XML file
tree = ET.parse("burma14.xml")
burma_root = tree.getroot()

In [838]:
# Find the graph element
graph = burma_root.find('graph')

In [839]:
# Initialize variables
burma_num_vertices = int(burma_root.find('description').text.split('-')[0])
burma_precision = int(burma_root.find('doublePrecision').text)
burma_cost_matrix = np.zeros((burma_num_vertices, burma_num_vertices))

In [840]:
# Initialize vertex_id
vertex_id = 0

In [841]:
# Iterate through the edge elements and populate the cost matrix
for vertex in graph.findall('vertex'):
    for edge in vertex.findall('edge'):
        target_vertex = int(edge.text)
        cost = float(edge.attrib['cost'])
        burma_cost_matrix[vertex_id, target_vertex] = cost

    # Increment vertex_id
    vertex_id += 1

# Print the cost matrix (optional)
print(burma_cost_matrix)

num_burma_cities = burma_num_vertices

[[   0.  153.  510.  706.  966.  581.  455.   70.  160.  372.  157.  567.
   342.  398.]
 [ 153.    0.  422.  664.  997.  598.  507.  197.  311.  479.  310.  581.
   417.  376.]
 [ 510.  422.    0.  289.  744.  390.  437.  491.  645.  880.  618.  374.
   455.  211.]
 [ 706.  664.  289.    0.  491.  265.  410.  664.  804. 1070.  768.  259.
   499.  310.]
 [ 966.  997.  744.  491.    0.  400.  514.  902.  990. 1261.  947.  418.
   635.  636.]
 [ 581.  598.  390.  265.  400.    0.  168.  522.  634.  910.  593.   19.
   284.  239.]
 [ 455.  507.  437.  410.  514.  168.    0.  389.  482.  757.  439.  163.
   124.  232.]
 [  70.  197.  491.  664.  902.  522.  389.    0.  154.  406.  133.  508.
   273.  355.]
 [ 160.  311.  645.  804.  990.  634.  482.  154.    0.  276.   43.  623.
   358.  498.]
 [ 372.  479.  880. 1070. 1261.  910.  757.  406.  276.    0.  318.  898.
   633.  761.]
 [ 157.  310.  618.  768.  947.  593.  439.  133.   43.  318.    0.  582.
   315.  464.]
 [ 567.  581.  374.  

### Parse the Brazil file extract the contents

In [842]:
# Parse the XML file
tree = ET.parse("brazil58.xml")
brazil_root = tree.getroot()

In [843]:
# Find the graph element
graph = brazil_root.find('graph')

In [844]:
# Initialize variables
brazil_num_vertices = int(brazil_root.find('description').text.split()[0])
brazil_precision = int(burma_root.find('doublePrecision').text)
brazil_cost_matrix = np.zeros((brazil_num_vertices, brazil_num_vertices))

In [845]:
# Initialize vertex_id
vertex_id = 0

In [846]:
# Iterate through the edge elements and populate the cost matrix
for vertex in graph.findall('vertex'):
    for edge in vertex.findall('edge'):
        target_vertex = int(edge.text)
        cost = float(edge.attrib['cost'])
        brazil_cost_matrix[vertex_id, target_vertex] = cost

    # Increment vertex_id
    vertex_id += 1

# Print the cost matrix (optional)
print(brazil_cost_matrix)

num_brazil_cities = brazil_num_vertices

[[   0. 2635. 2713. ... 3870. 1417.  739.]
 [2635.    0.  314. ... 2072. 1196. 1517.]
 [2713.  314.    0. ... 1882. 2699. 1557.]
 ...
 [3870. 2072. 1882. ...    0. 2328. 2986.]
 [1417. 1196. 2699. ... 2328.    0.  962.]
 [ 739. 1517. 1557. ... 2986.  962.    0.]]


### Generate a random Chromosome / Tour

In [847]:
#Generate random tour depending on the number of cities from each country
def generateTour(num_cities):
    tour = random.sample(range(1, num_cities + 1), num_cities)
    return tour

### Setting  Genetic Algorithm Parameters

In [848]:
population_size = 100 # Size of the population
mutation_rate = 0.1 # setting the mutation rate

In [849]:
generation = 0

### Generate a population 

In [850]:
# Generating based on the population size
def generateRandomPopulation(num_cities):
    population = []
    for _ in range(population_size):
        population.append(generateTour(num_cities))
    return population

### Evaluate Fitness Function


In [851]:
##Evaluating the fitness value for the individual gene
#def generatefitness(individual, precision, country):
#    global generation
#    
#    if country == "Burma" :
#        if generation > 0 :
#            cost_matrix = burma_cost_matrix;
#            cost = 0
#            for i in range(len(individual) - 1):
#                cost += cost_matrix[individual[i] - 1][individual[i + 1] - 1]
#            cost += cost_matrix[individual[-1] - 1][individual[0] - 1]
#            
#            generation -= 1
#            return round(1/cost, precision)
#        else :
#            raise TerminationException
#    elif country == "Brazil":
#        if generation > 0 :
#            cost_matrix = brazil_cost_matrix;
#            cost = 0
#            for i in range(len(individual) - 1):
#                cost += cost_matrix[individual[i] - 1][individual[i + 1] - 1]
#            cost += cost_matrix[individual[-1] - 1][individual[0] - 1]
#            
#            generation -= 1
#            return round(1/cost, precision)
#        else:
#             raise TerminationException  
        

In [852]:
def generatefitness(individual, precision, country):
    global generation 
    
    if generation > 0:
        if country == "Burma":
            cost_matrix = burma_cost_matrix
        elif country == "Brazil":
            cost_matrix = brazil_cost_matrix
        else:
            raise ValueError("Invalid country")
    
        cost = 0
        for i in range(len(individual) - 1):
            cost += cost_matrix[individual[i] - 1][individual[i + 1] - 1]
        cost += cost_matrix[individual[-1] - 1][individual[0] - 1]
        generation -= 1
        return round(1 / cost, precision)
    

### Function for Population Fitness

In [853]:
def generatePopulationFitness(population, precision,country):
    fitness = []
    for element in population:
        fitness.append(generatefitness(element,precision,country))
    return fitness

# population_fitness = GetPopulationFitness(population)
# print(population_fitness)

### Swap Mutation 

In [854]:
#Performing swap mutation on the input tour
def swapMutate(tour, num_cities):
    if random.random() < mutation_rate:
        # Swap two random cities in the tour
        idx1, idx2 = random.sample(range(num_cities), 2)
        tour[idx1], tour[idx2] = tour[idx2], tour[idx1]
    return tour

### Inversion Mutation

In [855]:
def inversionMutation(tour):
    # Select two random positions in the chromosome
    pos1, pos2 = sorted(random.sample(range(len(tour)), 2))
    print(pos1, pos2)
    # Perform inversion mutation
    segment_to_invert = tour[pos1:pos2 + 1]
    inverted_segment = segment_to_invert[::-1]

    # Update the chromosome with the inverted segment
    tour[pos1:pos2 + 1] = inverted_segment
    
    return tour

### Crossover Function

In [856]:
#Performing single point crossover using two parents
def singlePointCrossover(parent1, parent2):
    #Choosing a random point in the parent 
    point = np.random.randint(1,len(parent1))
    #Spliting the chromosome and adding the distinct part together
    parent1_left = parent1[:point]
    parent1_right = parent1[point:]
    parent2_left = parent2[:point]
    parent2_right = parent2[point:]
    child1 = parent1_left + parent2_right
    child2 = parent2_left + parent1_right
    #find missing and conflicting data in parent1 
    parent1_missing = [i for i in parent1 if i not in child1]
    parent1_conflict = [i for i in parent1_left if i in parent2_right]
    #fix parent1 by replacing the conflicting data with the missing elements
    for element in child1:
        if element in parent1_conflict:
            copy = element
            child1[child1.index(element)] = choice([i for i in parent1_missing if i not in child1])
            parent1_conflict.remove(copy)

    #find missing and conflicting data in parent2
    parent2_missing = [i for i in parent2 if i not in child2]
    parent2_conflict = [i for i in parent2_left if i in parent1_right]
    #fix parent2ee by replacing the conflicting data with the missing elements
    for element in child2:
        if element in parent2_conflict:
            copy = element
            child2[child2.index(element)] = choice([i for i in parent2_missing if i not in child2])
            parent2_conflict.remove(copy)
    
    return child1,child2

### Ordered Crossover Function

In [857]:
def orderedCrossover(parent1,parent2):
    size = len(parent1)

    # Choose random start/end position for crossover
    child1, child2 = [-1] * size, [-1] * size
    start, end = sorted([random.randrange(size) for _ in range(2)])

    # Replicate parent1's sequence for child1, parent2's sequence for child2
    child1_inherited = []
    child2_inherited = []
    for i in range(start, end + 1):
        child1[i] = parent1[i]
        child2[i] = parent2[i]
        child1_inherited.append(parent1[i])
        child2_inherited.append(parent2[i])

    print(child1, child2)
    #Fill the remaining position with the other parents' entries
    current_parent2_position, current_parent1_position = 0, 0

    fixed_pos = list(range(start, end + 1))       
    i = 0
    while i < size:
        if i in fixed_pos:
            i += 1
            continue

        test_child1 = child1[i]
        if test_child1==-1: 
            parent2_trait = parent2[current_parent2_position]
            while parent2_trait in child1_inherited:
                current_parent2_position += 1
                parent2_trait = parent2[current_parent2_position]
            child1[i] = parent2_trait
            child1_inherited.append(parent2_trait)
            
        test_child2 = child2[i]
        if test_child2==-1: 
            parent1_trait = parent1[current_parent1_position]
            while parent1_trait in child2_inherited:
                current_parent1_position += 1
                parent1_trait = parent1[current_parent1_position]
            child2[i] = parent1_trait
            child2_inherited.append(parent1_trait)

        i +=1

    return child1, child2

### Selection Functions

In [858]:
def tournament_selection(population,size, cost_matrix):
    tournament = []
    #Choose the number(size) from the population and add that to tournament list
    for i in range(size):
        tournament.append(choice([solution for solution in population]))
    #The winner of the tournament is the solution with the best fitness
    parent = max(tournament, key=lambda solution: generatefitness(solution, cost_matrix))
    return parent

In [859]:
def rankBasedSelection(population,pop_fitness):
    #Combine the population and its fitness
    fitness_population = sorted(zip(pop_fitness, population))
    #Sort the population and fitness
    sorted_population = [x for y,x in fitness_population]
    sorted_pop_fitness = [y for y,x in fitness_population]
    rank = []
    #Assign ranks to the solutions in the population
    for i in range(len(sorted_population)):
        rank.append(i+1)
    rank_total = sum(rank)
    probability = []
    #Find the probabilities of choosing a solution from the population based on its rank
    for r in rank:
        probability.append(r/rank_total)
    indeces = np.arange(len(sorted_population))
    #Choose 2 parents randomly based on the probabilities
    choice_1, choice_2 = np.random.choice(indeces,2,replace=True,p=probability)
    parent_1, parent_2 = sorted_population[choice_1], sorted_population[choice_2]
    return parent_1,parent_2
    

In [860]:
def rouletteWheelSelection(population, pop_fitness):
    total_fitness = sum(pop_fitness)
    
    # Select the first parent
    selection_point1 = random.uniform(0, total_fitness)
    cumulative_fitness = 0
    for i, fitness in enumerate(pop_fitness):
        cumulative_fitness += fitness
        if cumulative_fitness >= selection_point1:
            parent1 = population[i]
            break

    # Select the second parent
    selection_point2 = random.uniform(0, total_fitness)
    cumulative_fitness = 0
    for i, fitness in enumerate(pop_fitness):
        cumulative_fitness += fitness
        if cumulative_fitness >= selection_point2:
            parent2 = population[i]
            break

    return parent1, parent2

### Replacement Functions

In [861]:
def replaceWeakest(population, population_fitness, child, child_fitness, precision):
    #Search the worst fitness in the population and store it
    worst_fitness_ind = min(pop_fitness)
    #Replace the worst individual if the new candidate individual is better or the same
    if (child is not None and child_fitness is not None and population is not None and population_fitness is not None):
        if round(child_fitness,precision) >= round(worst_fitness_ind,precision):
            list_index = pop_fitness.index(worst_fitness_ind)
            del pop_fitness[list_index]
            del population[list_index]
            population.append(child)
            pop_fitness.append(child_fitness)
        return population, pop_fitness

### Average Fitness Of The Population

In [862]:
def calculateAverageFitness(population_fitness,size):
    return sum(population_fitness)/size

### Best Fitness Value 

In [863]:
def calculateBestSolution(population, population_fitness):
    best_fitness = max(population_fitness)
    list_index = population_fitness.index(best_fitness)
    return population[list_index],best_fitness

### TSP Default Parameter Values

#### Below are the default parameter values
1. Population size = 100
2. Tournament size = 10
3. Generation size = 10000
4. Mutation rate = 0.1
5. Precision = 15
6. Crossover function = Single point crossover
7. Mutation function = Swap Mutation
8. Replacement function = Replacement of weakest individual


## Experiment 1

#### We aim to determine the optimal selection technique in this experiment. We contrast tournament selection, rank-based selection, and fitness proportionate selection. Every other parameter remains unchanged, maintaining their default values. To fine-tune the tournament solution, account for variance in the population size and tournament size. 

### Calculate The Best Fitness Selection for Burma Data

In [864]:
# Lists to store data for plotting
average_fitness_history = []
best_fitness_history = []

for run in range(RUNS):
    POPULATION_SIZE = 100
    generation = 10000
    ts = str(time.time())
    ts = ts.replace(".","_")
    
    try:
        population = generateRandomPopulation(num_burma_cities)
        population_fitness = generatePopulationFitness(population, burma_precision, "Burma")
        while True:
                average_fitness = calculateAverageFitness(population_fitness,POPULATION_SIZE)
                average_fitness = str("{:."+str(burma_precision)+"f}").format(average_fitness)
                best_solution,best_fitness = calculateBestSolution(population,population_fitness)
                best_fitness = str("{:."+str(burma_precision)+"f}").format(best_fitness)
                best_solution = ' '.join(str(g) for g in best_solution)
                
                a,b = rouletteWheelSelection(population,population_fitness)
                c,d = singlePointCrossover(a,b)
                e = swapMutate(c, num_burma_cities)
                e_fitness = generatefitness(e,burma_precision,"Burma")
                f = swapMutate(d,num_burma_cities )
                f_fitness = generatefitness(f,burma_precision,"Burma")
                population,population_fitness = replaceWeakest(population, population_fitness, e, e_fitness, burma_precision)
                population,population_fitness = replaceWeakest(population, population_fitness, f, f_fitness, burma_precision)           
                
    except TerminationException:
        print("---TERMINATION CONDITION REACHED---")
    
    average_fitness_history.append(float(average_fitness))
    best_fitness_history.append(float(best_fitness))
    print(str("{:."+str(burma_precision)+"f}").format(calculateAverageFitness(population_fitness,POPULATION_SIZE)))
    print(calculateBestSolution(population,population_fitness))

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# Plotting the average fitness over generations
plt.figure(figsize=(10, 5))
plt.plot(range(len(average_fitness_history)), average_fitness_history, label='Average Fitness')
plt.title('Average Fitness Over Generations')
plt.xlabel('Generations')
plt.ylabel('Fitness')
plt.legend()
plt.show()

In [ ]:
# Plotting the best fitness over generations
plt.figure(figsize=(10, 5))
plt.plot(range(len(best_fitness_history)), best_fitness_history, label='Best Fitness')
plt.title('Best Fitness Over Generations')
plt.xlabel('Generations')
plt.ylabel('Fitness')
plt.legend()
plt.show()